<a href="https://colab.research.google.com/github/Abhishek603124/LDA-Topic-Modeling/blob/main/LDA-Topic-Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Remove any previously installed gensim versions (optional)
!pip uninstall -y gensim

# Install a compatible version of gensim and numpy
!pip install gensim==4.3.1 scipy==1.10.1 numpy==1.23.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
treescope 0.1.9 requires 

In [2]:
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 7.9 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import random
import spacy

from gensim import models, corpora
from gensim import similarities
from gensim.models.coherencemodel import CoherenceModel
from wordcloud import WordCloud

# First pass at building an LDA topic model for our corpus

We'll use a corpus of over 90,000 CNN news articles originally compiled for training question answering models. I lightly processed them to remove some metadata and put them on Google Drive.

In [4]:
# This code is a small trick to avoid annoying "encoding errors" when handling languages, emojis, or special characters
# — especially in Colab or cloud notebooks.
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
# It's very useful in Colab, where you might need to download large datasets or models stored in Google Drive.
!pip install --upgrade --no-cache-dir gdown

In [6]:
# Download the CNN corpus.
!gdown 'https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf'

Downloading...
From (original): https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf
From (redirected): https://drive.google.com/uc?id=122fC9XpNwFKx0ryRVKJz5MWUTzA3Vpsf&confirm=t&uuid=069220b8-ed45-4ae9-9eb3-871bab9cd30b
To: /content/cnn_articles.txt
100% 365M/365M [00:03<00:00, 95.7MB/s]


The corpus is one large text file with each article in the corpus separated by an @delimiter string. We'll split the articles and place them in a list.

In [7]:
with open('cnn_articles.txt', 'r', encoding='utf8') as f:
  articles = f.read().split('@delimiter')

In [8]:
print(len(articles))
print(articles[0])

92579
 -- Children in war-ravaged Afghanistan are safer than those growing up in London or New York, NATO's top civilian envoy says.

Mark Sedwill, the senior civilian representative for NATO, made the comments on an episode of CBBC's "Newsround," which is airing Monday.

In the show -- a BBC current-affairs program for children -- several youngsters in Kabul, Afghanistan, say they are afraid of daily violence and the frequent explosions in their war-torn country.

In response, Sedwill says: "Here in Kabul, and other big cities actually, there are very few of these bombs. The children are probably safer here than they would be in London, New York or Glasgow or many other cities.

"Most children can go about their lives in safety. It's a very family-oriented society. So it is a little bit like a city of villages," he added.

A U.N. report released earlier this year seems to contradict Sedwill's assessment.

The February report, by the special representative for children and armed confli

In [9]:
dataset = articles[:20000]

We'll filter out punctuations, newlines, and any tokens containing non-alphabetic characters.

In [10]:
nlp = spacy.blank('en')

def basic_filter(tokenized_doc):
  return [t.text for t in tokenized_doc if
          not t.is_punct and \
          not t.is_space and \
          t.is_alpha]

In this demo, we'll leverage spaCy's nlp.pipe function which can process a corpus as a batch (or a series of batches) and use multiple processes. Here, we'll process our dataset as a batch across multiple processes, then run the tokenized doc objects through the basic_filter function. You can adjust NUM_PROCESS as you wish.

In [11]:
NUM_PROCESS = 4

In [12]:
# this is a kind of pipeline to perform all the basic preprocessing step from the above function with n_process = no. of parallel processes done at one time
tokenized_articles = list(map(basic_filter,nlp.pipe(dataset, n_process = NUM_PROCESS)))

In [20]:
print(tokenized_articles[0])

['Children', 'in', 'war', 'ravaged', 'Afghanistan', 'are', 'safer', 'than', 'those', 'growing', 'up', 'in', 'London', 'or', 'New', 'York', 'NATO', 'top', 'civilian', 'envoy', 'says', 'Mark', 'Sedwill', 'the', 'senior', 'civilian', 'representative', 'for', 'NATO', 'made', 'the', 'comments', 'on', 'an', 'episode', 'of', 'CBBC', 'Newsround', 'which', 'is', 'airing', 'Monday', 'In', 'the', 'show', 'a', 'BBC', 'current', 'affairs', 'program', 'for', 'children', 'several', 'youngsters', 'in', 'Kabul', 'Afghanistan', 'say', 'they', 'are', 'afraid', 'of', 'daily', 'violence', 'and', 'the', 'frequent', 'explosions', 'in', 'their', 'war', 'torn', 'country', 'In', 'response', 'Sedwill', 'says', 'Here', 'in', 'Kabul', 'and', 'other', 'big', 'cities', 'actually', 'there', 'are', 'very', 'few', 'of', 'these', 'bombs', 'The', 'children', 'are', 'probably', 'safer', 'here', 'than', 'they', 'would', 'be', 'in', 'London', 'New', 'York', 'or', 'Glasgow', 'or', 'many', 'other', 'cities', 'Most', 'children

To start off, we'll go with 20 topics. With most topic models including LDA, there isn't a clear recipe on how to pick the optimal number of topics. The nature and composition of the data (e.g. average length of each document) has a major impact on how many topics are interpretable by a human. Often, it's best to go with something reasonable to begin with and then try different topic numbers.

For this corpus, I'm going with 20 topics which is a small amount relative to the corpus size, but my reasoning is that since this is a general mainstream news corpus, the topics themselves are going to be fairly broad.

In [13]:
NUM_TOPICS = 20

After tokenizing our text, the first step with Gensim is to construct a Dictionary mapping words to integer IDs.

In [14]:
# build a dictionary of word<--->id mappings
dictionary = corpora.Dictionary(tokenized_articles)
sample_token = 'news'
print(f'Id for \'{sample_token}\' token: {dictionary.token2id[sample_token]}')

Id for 'news' token: 1039


Builds a dictionary mapping each unique word in tokenized_articles to a unique ID.

Retrieves and prints the ID assigned to the word 'news'.

In [15]:
# it converts the list of words into bag of words i.e. assigns the count of unique ids i.e. how many time that word appeared like (word_id, frequency)
corpus_bow = [dictionary.doc2bow(article) for article in tokenized_articles]

In [16]:

lda_model = models.LdaModel(corpus=corpus_bow,
                            id2word=dictionary,
                            num_topics=NUM_TOPICS,
                            random_state = 1)

Once our model is generated, we can view the topics inferred. By default, the model's print_topics method shows the top 20 topics and each topic's ten most significant words.

In [17]:
lda_model.print_topics()

[(0,
  '0.068*"the" + 0.041*"of" + 0.027*"and" + 0.025*"to" + 0.025*"in" + 0.023*"a" + 0.013*"is" + 0.009*"for" + 0.008*"that" + 0.008*"The"'),
 (1,
  '0.046*"the" + 0.031*"I" + 0.030*"a" + 0.028*"to" + 0.022*"and" + 0.019*"in" + 0.017*"of" + 0.012*"was" + 0.012*"it" + 0.012*"that"'),
 (2,
  '0.064*"the" + 0.030*"of" + 0.025*"to" + 0.023*"and" + 0.020*"in" + 0.017*"a" + 0.011*"that" + 0.011*"is" + 0.009*"Syria" + 0.008*"for"'),
 (3,
  '0.038*"the" + 0.032*"to" + 0.029*"and" + 0.029*"of" + 0.024*"a" + 0.017*"that" + 0.016*"in" + 0.013*"is" + 0.010*"are" + 0.009*"for"'),
 (4,
  '0.069*"the" + 0.030*"of" + 0.029*"to" + 0.024*"and" + 0.021*"in" + 0.019*"a" + 0.015*"said" + 0.011*"The" + 0.010*"that" + 0.009*"on"'),
 (5,
  '0.060*"the" + 0.031*"in" + 0.029*"of" + 0.028*"to" + 0.027*"and" + 0.021*"a" + 0.014*"said" + 0.011*"that" + 0.011*"The" + 0.008*"is"'),
 (6,
  '0.054*"the" + 0.033*"to" + 0.030*"of" + 0.027*"in" + 0.026*"and" + 0.025*"a" + 0.012*"said" + 0.011*"that" + 0.009*"The" + 0.0

#Improving preprocessing for better results.
*For our next attempt, we'll*

remove stop words using the default spaCy stopword list. Given this is a corpus of news articles, there may be other stop words to consider such as salutations ("Mr", "Mrs"), and words related to quotes and thoughts ("say", "think"). But for this, we'll stick to defaults unless we see reason to do otherwise.
consider only the words the spaCy tagger flags as nouns, verbs, and adjectives. Including words with only certain POS tags is a common approach to improving topic models.
take the lemma.

In [18]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

def improved_filter(tokenized_doc):
  return [t.lemma_ for t in tokenized_doc if
          t.is_alpha and\
          not t.is_punct and \
          not t.is_space and \
          not t.is_stop and \
          t.pos_ in ['NOUN','VERB','ADJ']]

In [19]:
# We'll need to retokenize everything and rebuild the BOWs. Because we're now
# using the POS tagger, this will take longer. The "w_pos" in the variable
# names below just means "with part-of-speech".
tokenized_articles_w_pos = list(map(improved_filter, nlp.pipe(dataset, n_process=NUM_PROCESS)))
dictionary_w_pos = corpora.Dictionary(tokenized_articles_w_pos)
corpus_bow_w_pos = [dictionary_w_pos.doc2bow(article) for article in tokenized_articles_w_pos]

In [20]:
lda_model = models.LdaModel(corpus=corpus_bow_w_pos, num_topics=NUM_TOPICS, id2word=dictionary_w_pos, random_state=1)

In [21]:
lda_model.print_topics()

[(0,
  '0.016*"country" + 0.015*"government" + 0.014*"say" + 0.011*"political" + 0.010*"election" + 0.008*"leader" + 0.007*"group" + 0.007*"state" + 0.007*"vote" + 0.007*"party"'),
 (1,
  '0.027*"say" + 0.010*"water" + 0.008*"city" + 0.008*"people" + 0.007*"area" + 0.006*"year" + 0.005*"mile" + 0.005*"home" + 0.004*"foot" + 0.004*"day"'),
 (2,
  '0.020*"say" + 0.015*"storm" + 0.009*"year" + 0.009*"car" + 0.007*"people" + 0.007*"report" + 0.007*"power" + 0.006*"state" + 0.006*"wind" + 0.006*"high"'),
 (3,
  '0.020*"film" + 0.019*"say" + 0.015*"movie" + 0.009*"year" + 0.009*"church" + 0.006*"world" + 0.006*"star" + 0.005*"people" + 0.004*"day" + 0.004*"come"'),
 (4,
  '0.022*"say" + 0.021*"israeli" + 0.012*"palestinian" + 0.009*"space" + 0.009*"british" + 0.009*"year" + 0.009*"visit" + 0.008*"royal" + 0.007*"peace" + 0.007*"diamond"'),
 (5,
  '0.057*"say" + 0.013*"police" + 0.012*"report" + 0.011*"official" + 0.010*"government" + 0.009*"people" + 0.009*"attack" + 0.009*"kill" + 0.008*"au

# Trimming low- and high-frequency words.




In [22]:
# The size of the dictionary before filtering.
len(dictionary_w_pos)

39601

In [23]:
# it checks the number of documents in which a word appears, not just the frequency of the word in each individual document.
# It helps to filter out rare words (which may be typos or irrelevant) and overused words (which don't contribute meaningful distinctions between topics)
dictionary_w_pos.filter_extremes(no_below=10,no_above=0.5)

In [24]:
# The size of the dictionary after filtering.
len(dictionary_w_pos)

13802

In [25]:
# Rebuild bag of words.
corpus_bow_w_pos_filtered = [dictionary_w_pos.doc2bow(article) for article in tokenized_articles_w_pos]

This time, we're passing additional arguments when building the model. alpha is the prior on the document-topic distribution, and eta is the prior on the topic-word distribution (this was beta in the slides), and passes is the number of complete passes through the corpus during training.

In [28]:
lda_model = models.ldamodel.LdaModel(corpus=corpus_bow_w_pos_filtered,
                                     id2word=dictionary_w_pos,
                                     num_topics=NUM_TOPICS,
                                     passes=10,
                                     alpha='auto',
                                     eta='auto',
                                     random_state=1)

In [29]:
lda_model.print_topics()

[(0,
  '0.018*"country" + 0.014*"world" + 0.008*"nation" + 0.007*"korean" + 0.007*"church" + 0.007*"history" + 0.007*"american" + 0.007*"war" + 0.007*"leader" + 0.006*"visit"'),
 (1,
  '0.021*"think" + 0.017*"go" + 0.013*"good" + 0.013*"know" + 0.012*"world" + 0.011*"want" + 0.011*"win" + 0.011*"get" + 0.009*"thing" + 0.009*"sport"'),
 (2,
  '0.016*"company" + 0.011*"oil" + 0.009*"business" + 0.008*"money" + 0.008*"market" + 0.008*"country" + 0.007*"new" + 0.007*"world" + 0.007*"economic" + 0.007*"pay"'),
 (3,
  '0.025*"flight" + 0.024*"plane" + 0.019*"ship" + 0.019*"passenger" + 0.015*"crash" + 0.014*"airport" + 0.013*"fly" + 0.012*"crew" + 0.012*"pilot" + 0.012*"airline"'),
 (4,
  '0.020*"election" + 0.017*"political" + 0.015*"vote" + 0.014*"party" + 0.014*"president" + 0.013*"campaign" + 0.011*"candidate" + 0.010*"state" + 0.009*"presidential" + 0.009*"government"'),
 (5,
  '0.015*"government" + 0.013*"official" + 0.012*"report" + 0.009*"law" + 0.008*"statement" + 0.008*"state" + 0.

With improved filtering and low- and high-frequency words trimmed, we can see the topic-word distributions containing certain themes such as crime, travel, entertainment, etc.

In [31]:
print(lda_model.alpha)
print(lda_model.eta)

[0.09153689 0.10631368 0.09982678 0.05782061 0.07625443 0.13713677
 0.12765585 0.0708163  0.13474113 0.09373292 0.07978228 0.09267671
 0.04758471 0.06228654 0.09462486 0.12749858 0.09986711 0.06018009
 0.08419022 0.10056212]
[0.08991699 0.0638742  0.04840612 ... 0.03929042 0.03929042 0.03929042]


We can look at the topic distribution comprising a given article using the model's get_document_topics method.

In [32]:
article_idx = 0
print(dataset[article_idx][:300])

 -- Children in war-ravaged Afghanistan are safer than those growing up in London or New York, NATO's top civilian envoy says.

Mark Sedwill, the senior civilian representative for NATO, made the comments on an episode of CBBC's "Newsround," which is airing Monday.

In the show -- a BBC current-affa


In [34]:
# return topic distribution for an article sorted by probability.
#  gives you a list of topics for that article from most dominant to least dominant based on the probability.
topics = sorted(lda_model.get_document_topics(corpus_bow_w_pos_filtered[article_idx]), key = lambda tup : tup[1])[::-1]
topics

[(9, 0.48490885),
 (6, 0.20118444),
 (19, 0.14771438),
 (13, 0.077252746),
 (7, 0.042535994),
 (8, 0.022521636),
 (18, 0.015828997)]

 To get the top words representing topic

In [35]:
# See the top words for the most dominant topic
lda_model.show_topic(topics[0][0])

[('military', 0.01951611),
 ('attack', 0.017634671),
 ('government', 0.014643469),
 ('force', 0.014076337),
 ('group', 0.012214274),
 ('kill', 0.010812059),
 ('official', 0.00985792),
 ('country', 0.008947157),
 ('security', 0.008664641),
 ('war', 0.008643991)]

In [36]:
# View the words of the second-most prevalent topic from the previous article.
lda_model.show_topic(topics[1][0])

[('family', 0.032311738),
 ('child', 0.029841105),
 ('mother', 0.013546273),
 ('life', 0.013307522),
 ('old', 0.01305365),
 ('know', 0.012495292),
 ('son', 0.011130354),
 ('girl', 0.010982933),
 ('father', 0.010693691),
 ('parent', 0.010431822)]

The function below takes a document index and returns a DataFrame containing:

the topics comprising the document up to a minimum probability.
the top words of each topic.

In [37]:
def get_top_topics(article_idx, min_topic_prob):

  # Sort from highest to lowest topic probability.
  topic_prob_pairs = sorted(lda_model.get_document_topics(corpus_bow_w_pos_filtered[article_idx],
                                                          minimum_probability=min_topic_prob),
                            key=lambda tup: tup[1])[::-1]

  word_prob_pairs = [lda_model.show_topic(pair[0]) for pair in topic_prob_pairs]
  topic_words = [[pair[0] for pair in collection] for collection in word_prob_pairs]

  data = {
      'Major Topics': topic_prob_pairs,
      'Topic Words': topic_words
  }

  return pd.DataFrame(data)


In [38]:
pd.set_option('max_colwidth', 600)
snippet_length = 300
min_topic_prob = 0.25

article_idx = 1
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)


 -- The Hard Rock Cafe franchise, famed for its rock'n'roll memorabilia, is expanding its reach into hotels and casinos -- but its chief executive says the brand will remain "special."

"We don't want to be in every street corner, in every city of the UK," Hard Rock's Hamish Dodds told CNN.

"We wa


,Major Topics,Topic Words
0,"(18, 0.34388027)","[city, hotel, place, travel, local, world, room, restaurant, building, visitor]"
1,"(2, 0.31973666)","[company, oil, business, money, market, country, new, world, economic, pay]"


In [39]:
article_idx = 10
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)


 -- The House Republican leadership upped the ante Thursday in the ongoing debate over the size and scope of the federal budget, unveiling a proposal to cut spending by $375 billion over the next five years.

House GOP Minority Whip Eric Cantor has been an outspoken critic of Democratic budget prop


,Major Topics,Topic Words
0,"(11, 0.590642)","[issue, bill, law, right, state, health, policy, debate, care, think]"


In [40]:
article_idx = 100
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)


 -- A tip led to Wednesday's arrest in Louisiana of a suspected serial bank robber dubbed the "Granddad Bandit," federal authorities said.

Michael Francis Mara, 52, was captured at his home in Baton Rouge, FBI Special Agent Sheila Thorne told CNN.

The FBI went to the residence to execute an arres


,Major Topics,Topic Words
0,"(16, 0.5208886)","[police, officer, authority, man, report, kill, arrest, gun, shoot, accord]"


In [41]:
article_idx = 1000
print(dataset[article_idx][:snippet_length])
get_top_topics(article_idx, min_topic_prob)


 -- Desperate times call for desperate measures.

After a humbling League Cup defeat at the hands of lowly Sunderland, and with the reigning English champions a distant seventh in the Premier League table, Manchester United have broken the bank to bring in Juan Mata from Chelsea.

A club record fee


,Major Topics,Topic Words
0,"(17, 0.66084397)","[win, player, game, play, team, match, second, season, club, minute]"


The results of this model look the best so far and we can see a human-interpretable link between the distribution of topics in a document, the distribution of words in each topic, and the content of the document itself.

# Evaluation and Visualization

# Measuring topic models with coherence.
If a topic is a mixture of particular words, then one way to measure how semantically coherent a topic is to calculate co-occurrence among the words. That is, how often the top words in a topic co-occur together among the documents versus how often they occur independently.

Gensim's Coherence Model offers coherence implemented as a pipeline:

In [42]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_articles_w_pos, dictionary=dictionary_w_pos, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  -1.5878381133644321
